# Hurdle 3
## Introduction

DARPA will score all Hurdle 3 submissions by running participant provided code against DARPA provided test stimulus and scoring code. Both participant code and DARPA provided code will be encapsulated in the LXC container format. See here for an introduction to LXC:

https://linuxcontainers.org/lxc/introduction/ 

This repository includes a build script for generating the DARPA Scoring Container and an example Solution Container. The **Scoring Container** is provided by DARPA and includes the test stimulus and scoring code. The **Solution Container** includes a stub solution which shows an example of how to interact with the Scoring Container. 

When evaluating whether a participant's submitted solution passes or fails Hurdle 3, DARPA will use a Scoring Container built using the build_hurdle_3_trusty.sh script provided in this directory. The only difference will be in the particular transition matrices used for DARPA's state machine in each trial.

## Environment
See the document entitled LXC_Configuration_Instructions.ipynb at the root of the Phase 1 Hurdles repository for instructions on how to configure your Ubuntu host to run LXC containers.

## Building the LXC Containers

To build both the Scoring Container and the example Solution Container for Hurdle 3, run the following command from inside your cloned copy of the phase1-hurdles repository:

```console
user@host:~/phase1-hurdles/hurdle3$ ./build_hurdle_3_trusty.sh
```

This script will create a new lxc subdirectory in your current directory, containing two containers. Assuming the build script was run from ~/phase1-hurdles/hurdle3, there will be containers in:

~/phase1-hurdles/hurdle3/lxc/hurdle3_scoring_trusty_lxc 

and:

~/phase1-hurdles/hurdle3/lxc/hurdle3_solution_trusty_lxc. 

To run any of the standard lxc- commands such as lxc-attach, you will need to specify the absolute path to the lxc directory as the -P prefix parameter. 

## Starting the containers

Before you can use the containers, you must start each of them. 

The -P parameter specifies where LXC will look for containers. -n specifies the name of the container. This example assumes you have the containers in ~/phase1-hurdles/hurdle3/lxc and are currently in the ~/phase1-hurdles/hurdle3 directory. To run both containers, use the following commands:

First store the LXC path to a shell variable:

```console
user@host:~/phase1-hurdles/hurdle3$ LXC_PATH=$(pwd)/lxc
```

Then start both containers:


```console
user@host:~/phase1-hurdles/hurdle3$ lxc-start -P "$LXC_PATH" -n hurdle3_scoring_trusty_lxc
user@host:~/phase1-hurdles/hurdle3$ lxc-start -P "$LXC_PATH" -n hurdle3_solution_trusty_lxc
```

## Running the example hurdle solution against the DARPA scoring container

You can SSH into each container now that they are both running. To get the IP address of the scoring container, use 

```console
user@host:~/phase1-hurdles/hurdle3$ lxc-info -P "$LXC_PATH" -n hurdle3_scoring_trusty_lxc -iH
```

For this example, lets assume that command returned 10.0.3.158. 

To get the IP address of the solution container, use

```console
user@host:~/phase1-hurdles/hurdle3$ lxc-info -P "$LXC_PATH" -n hurdle3_solution_trusty_lxc -iH
```

For this example, lets assume that command returned 10.0.3.131. 

Our scoring container is then running at 10.0.3.158, and our solution container is running at 10.0.3.131. 

SSH into the **solution** container as the user sc2 with:

```console
user@host:~/phase1-hurdles/hurdle3$ ssh sc2@10.0.3.131
```

If this is the first time connecting to the container, you'll see a message similar to: 

```console
The authenticity of host '10.0.3.131 (10.0.3.131)' can't be established.
ECDSA key fingerprint is SHA256:5vkTtMcNxVQPKhTuIZhifJrwxWoUCVqmguvl6Ot4ZV0.
Are you sure you want to continue connecting (yes/no)?
```

Answer **yes**. 


You'll then be prompted for a password. Use: **sc2**


You should then see: 
```console
Welcome to Ubuntu 14.04.5 LTS (GNU/Linux 4.4.0-38-generic x86_64)

 * Documentation:  https://help.ubuntu.com/

The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.

sc2@hurdle3_solution_trusty_lxc:~$
```

Change to the hurdle 3 directory and start the solution server with no seed using the following commands:

```console
sc2@hurdle3_solution_trusty_lxc:~$ cd ~/phase1-hurdles/hurdle3
sc2@hurdle3_solution_trusty_lxc:~/phase1-hurdles/hurdle3$ ./Hurdle3SolutionServer.py
```

You should see the server print out 

```console
Starting the server...
```

In a new terminal prompt on your host machine, SSH to the **Scoring** container as user sc2 with:

```console
user@host:~/phase1-hurdles/hurdle3$ ssh sc2@10.0.3.158
```

If this is the first time connecting to the container, you'll see a message similar to: 

```console
The authenticity of host '10.0.3.158 (10.0.3.158)' can't be established.
ECDSA key fingerprint is SHA256:5vkTtMcNxVQPKhTuIZhifJrwxWoUCVqmguvl6Ot4ZV0.
Are you sure you want to continue connecting (yes/no)? 
```

Answer **yes**. 


You'll then be prompted for a password. Use: **sc2**


You should then see: 
```console
Welcome to Ubuntu 14.04.5 LTS (GNU/Linux 4.4.0-38-generic x86_64)

 * Documentation:  https://help.ubuntu.com/

The programs included with the Ubuntu system are free software;
the exact distribution terms for each program are described in the
individual files in /usr/share/doc/*/copyright.

Ubuntu comes with ABSOLUTELY NO WARRANTY, to the extent permitted by
applicable law.

sc2@hurdle3_scoring_trusty_lxc:~$ 
```

Change to the hurdle 3 directory and start the scoring application. Note that you will need to tell the scoring application the IP address of the **Solution Container**, in this case 10.0.3.131. Start up the scoring application with no seed using the following commands:

```console
sc2@hurdle3_scoring_trusty_lxc:~$ cd ~/phase1-hurdles/hurdle3
sc2@hurdle3_scoring_trusty_lxc:~/phase1-hurdles/hurdle3$ ./Hurdle3Scoring.py --host=10.0.3.131
```

The Hurdle3Scoring application will execute a default of 10 trials, each with a default of 30000 rounds. It will compute an average score per round, compare that against a threshold, and declare whether each trial is a pass or fail. It will then count the number of trials that passed and compare that against a threshold. 

Solutions must pass at least **6** out of 10 trials in order to pass Hurdle 3. Detailed results will be written out to a file, whos location can be specified to Hurdle3Scoring.py using the --result-file parameter. Run 

```console
sc2@hurdle3_scoring_trusty_lxc:~/phase1-hurdles/hurdle3$ ./Hurdle3Scoring.py --help
```

for a full list of command line arguments to Hurdle3Scoring.